The weekly income at Pickens Memorial Hospital depends on the number of patients admitted in the three separate
categories: medical, surgical, and pediatric. The hospital can admit a total of 100 medical patients, 80 surgical patients,
and 60 pediatric patients each week. However, because Pickens Memorial serves a large community, patient demand in
each category by itself exceeds the total patient capacity.
• Due to fixed overhead, the income per patient in each category actually increases as the number of patients increases.
Further, some patients who are initially classified as medical patients then get reclassified as surgical patients. As a
result, the income per surgical patient also depends on the number of medical patients admitted. The accountants at
Pickens Memorial have analyzed the situation, and have identified the following information:
– Income contribution per medical patient: = $120 + $8M
– Income contribution per surgical patient: = $200 + $10S + $3M
– Income contribution per pediatric patient: = $95 + $9P
– Where M = number of medical patients admitted; S = number of surgical patients admitted; and P = number of pediatric patients admitted
• Perkins Memorial identified three main constraints for this model: scanning capacity (x-rays, MRI, and CT scans),
surgical rooms capacity, and lab capacity. Table below shows the relevant weekly data for these three constraints for
each category of patient. The table also shows the weekly availabilities of each of these resources: total number of
scanning and surgical procedures is 420 and 90, respectively, and total hours of testing is 1150 hours.
Decisions
Number of
scannings
per Patient
Number of
Surgical
Procedures
per Patient
Number of
Lab Tests
per Patient
Medical patients 1 2
Surgical patients 3 1.15 2.5
Pediatric patients 2 2
Total resource availability 420 90 1150
• The hospital’s chief laboratory supervisor has noted that the required time per lab test increases as the total number of
patients increases. Based on historical data, the supervisor estimates this relationship to be as follows:
– Time required per lab test (in hours) = 2 + 0.001(M + S + P)
• Develop and solve an NLP model that identifies the optimal number of patients to be admitted to the Pickens Memorial
Hospital that maximizes the total weekly income in the hospital.

In [4]:
import pandas as pd
import gurobipy as gp
import numpy as np
from gurobipy import Model, GRB
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [5]:
#Load data
pickens_data = pd.read_excel("NLP.xlsx", sheet_name="Pickens", header=None)

In [6]:
pickens_data

,0,1,2,3,4,5,6,7
0,Pickens Memorial Hospital,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Decisions,NaN,Value,Income Contribution,Total Capacity per Patient Category,Number of scannings per Patient,Number of Surgical Procedures per Patient,Number of Lab Tests per Patient
2,Medical patients,M =,1,128,100,1,NaN,2
3,Surgical patients,S =,1,213,80,3,1.15,2.5
4,Pediatric patients,P =,1,104,60,2,NaN,2
5,Total resource availability,NaN,NaN,NaN,NaN,420,90,1150
6,Time required per lab test (in hours),NaN,2.003,NaN,NaN,NaN,NaN,NaN


In [8]:
# Create the Gurobi model
model = gp.Model("Pickens_Memorial_Hospital")

"""set the upperbond to the constraint on the decision variable"""" 
# Decision Variables: Number of admitted patients (Continuous for NLP)
M = model.addVar(lb=0, ub=100, vtype=GRB.CONTINUOUS, name="Medical_Patients")
S = model.addVar(lb=0, ub=80, vtype=GRB.CONTINUOUS, name="Surgical_Patients")
P = model.addVar(lb=0, ub=60, vtype=GRB.CONTINUOUS, name="Pediatric_Patients")

# Objective Function: Maximize Total Weekly Income
income = ((120 + 8*M) * M) + ((200 + 10*S + 3*M) * S) + ((95 + 9*P) * P)
model.setObjective(income, GRB.MAXIMIZE)

# Constraints

# 1. Scanning Capacity: M + 3S + 2P ≤ 420
model.addConstr(M + 3*S + 2*P <= 420, "Scanning_Capacity")

# 2. Surgical Room Capacity: 1.15S ≤ 90
model.addConstr(1.15*S <= 90, "Surgical_Room_Capacity")

# 3. Lab Testing Capacity (Nonlinear)
lab_time_per_test = 2 + 0.001 * (M + S + P)
total_lab_hours = lab_time_per_test * (2*M + 2.5*S + 2*P)
model.addConstr(total_lab_hours <= 1150, "Lab_Capacity")

# Allow non-convex NLP
model.Params.NonConvex = 2
"""Setting NonConvex = 2 tells Gurobi to use its specialized algorithms
to find the global optimum for problems where the functions 
(like your objective and constraints) are non-convex (which is
often the case for complex NLPs), although in this specific 
example, the objective function is a concave quadratic which 
simplifies things. However, the Lab Constraint's impact on the 
feasible region determines the overall convexity of the problem."""

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Medical Patients: {M.X:.2f}")
    print(f"Optimal Surgical Patients: {S.X:.2f}")
    print(f"Optimal Pediatric Patients: {P.X:.2f}")
    print(f"Maximum Weekly Income: ${model.ObjVal:.2f}")


Set parameter LicenseID to value 2606765
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1250U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
NonConvex  2

Optimize a model with 2 rows, 3 columns and 4 nonzeros
Model fingerprint: 0x09d7a86a
Model has 4 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [2e-03, 5e-03]
  QLMatrix range   [4e+00, 5e+00]
  Objective range  [1e+02, 2e+02]
  QObjective range [6e+00, 2e+01]
  Bounds range     [6e+01, 1e+02]
  RHS range        [9e+01, 4e+02]
  QRHS range       [1e+03, 1e+03]
Presolve removed 1 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 